In [93]:
import streamlit as st
import streamlit.components.v1 as components
import csv
import string
import os
import pickle
import pandas as pd
from datetime import datetime as dt
from base64 import b64encode
#import proximitypyhash as pph
#import pygeohash as gh
from streamlit_folium import st_folium
from streamlit_folium import folium_static
import folium
import numpy as np

from agile.filtering import query_location, query_date, query_adid, query_node
from agile.mapping import data_map 
from agile.locations import locations_of_interest
from agile.people import colocation
from agile.prediction import double_cluster, get_top_N_clusters, haversine
from agile.utils.tag import find_all_nearby_nodes
from agile.utils.geocode import reverse_geocode
from agile.utils.files import find, random_line, save, random_name
from agile.utils.dataframes import modify_and_sort_columns
from agile.profile import Profile
from agile.report import Report
from agile.centrality import compute_top_centrality
from agile.overview import adid_value_counts

from streamlit_option_menu import option_menu
import pygeohash as gh

from scipy.spatial import cKDTree
from math import radians, cos, sin, sqrt, pi

import geopandas as gpd
from shapely.geometry import Point, Polygon

In [8]:
with open('saved_data/saved_df.pkl', 'rb') as file:
    data = pickle.load(file)

In [9]:
adid = '21191f02-c141-e3f6-901e-f0c9222be423'

In [28]:
df = data[data['advertiser_id'] == adid]
df.head(3)

,Unnamed: 0,advertiser_id,datetime,latitude,longitude,horizontal_accuracy,carrier,model,wifi_ssid,wifi_bssid,geohash
79804,79804,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:55:36,46.277680,-119.194260,3.21,55,iPhone 4,NaN,NaN,c25pr8t7h2
80717,80717,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:17,46.277570,-119.194200,4.77,55,iPhone 4,NaN,NaN,c25pr8t6mu
80835,80835,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:23,46.277494,-119.194231,4.44,55,iPhone 4,NaN,NaN,c25pr8t3vn


In [11]:
data_map(data=df)

In [42]:
def create_folium_map(center_lat, center_lon, corners):
    """
    Create a Folium map with a marker at the center and a polygon representing the square.

    Parameters:
    - center_lat (float): Latitude of the map center.
    - center_lon (float): Longitude of the map center.
    - corners (list): List of four tuples representing the coordinates of the square corners.

    Returns:
    - Folium Map object.
    """
    # Create a Folium map centered around the given coordinates
    my_map = folium.Map(location=[center_lat, center_lon], zoom_start=15)

    # Add a marker for the center point
    folium.Marker(location=[center_lat, center_lon], popup='Center').add_to(my_map)

    # Create a polygon using the corner coordinates
    folium.Polygon(locations=corners, color='blue', fill=True, fill_opacity=0.3).add_to(my_map)

    return my_map

In [62]:
def get_square_corners(lat, lon, distance):
    """
    Calculate the coordinates of the corners of a square around a given point.

    Parameters:
    - lat (float): Latitude of the center point.
    - lon (float): Longitude of the center point.
    - distance (float): Distance from the center to each side of the square (in kilometers).

    Returns:
    - List of four tuples, each representing the (latitude, longitude) of a corner.
    """
    # Radius of the Earth in kilometers
    R = 6371.0

    # Convert latitude and longitude from degrees to radians
    lat_rad = radians(lat)
    lon_rad = radians(lon)

    # Calculate the change in coordinates
    delta_lat = (distance / R) * (180 / pi)
    delta_lon = (distance / R) * (180 / pi) / cos(lat * (pi / 180))

    # Calculate the coordinates of the square corners
    top_left = (lat + delta_lat, lon - delta_lon)
    top_right = (lat + delta_lat, lon + delta_lon)
    bottom_left = (lat - delta_lat, lon - delta_lon)
    bottom_right = (lat - delta_lat, lon + delta_lon)

    return [top_left, top_right, bottom_right, bottom_left]

In [63]:
test_lat, test_long = df.iloc[0][['latitude','longitude']].values

In [66]:
test_corners = get_square_corners(test_lat, test_long, .25)
test_corners

[(46.279928304014796, -119.1975129240934),
 (46.279928304014796, -119.1910070759066),
 (46.2754316959852, -119.1910070759066),
 (46.2754316959852, -119.1975129240934)]

In [67]:
create_folium_map(test_lat, test_long, test_corners)

In [74]:
lats_and_longs = []
for _, row in df.iterrows():
    lats_and_longs.append((row['latitude'],row['longitude']))

In [79]:
distance = .25
all_corners = []
for pair in lats_and_longs:
    all_corners += get_square_corners(pair[0], pair[1], distance)

In [80]:
create_folium_map(test_lat, test_long, all_corners)

In [81]:
df

,Unnamed: 0,advertiser_id,datetime,latitude,longitude,horizontal_accuracy,carrier,model,wifi_ssid,wifi_bssid,geohash
79804,79804,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:55:36,46.277680,-119.194260,3.21,55,iPhone 4,NaN,NaN,c25pr8t7h2
80717,80717,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:17,46.277570,-119.194200,4.77,55,iPhone 4,NaN,NaN,c25pr8t6mu
80835,80835,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:23,46.277494,-119.194231,4.44,55,iPhone 4,NaN,NaN,c25pr8t3vn
81142,81142,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:36,46.276686,-119.193810,4.65,55,iPhone 4,NaN,NaN,c25pr8mtnz
81295,81295,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 07:56:43,46.276163,-119.193635,1.18,55,iPhone 4,NaN,NaN,c25pr8mf4y
...,...,...,...,...,...,...,...,...,...,...,...
798258,798258,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 14:56:56,46.209244,-119.013110,1.13,55,iPhone 4,NaN,NaN,c25qgu1k3e
798345,798345,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 14:56:59,46.209063,-119.012800,6.30,55,iPhone 4,NaN,NaN,c25qgu1e2c
798397,798397,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 14:57:02,46.208883,-119.012500,5.37,55,iPhone 4,NaN,NaN,c25qgu1drb
798509,798509,21191f02-c141-e3f6-901e-f0c9222be423,2018-04-08 14:57:06,46.208640,-119.012228,1.84,55,iPhone 4,NaN,NaN,c25qgu1by4


In [88]:
valid_geohashes = df['geohash'].unique()

In [92]:
data.query("geohash in @valid_geohashes")

,Unnamed: 0,advertiser_id,datetime,latitude,longitude,horizontal_accuracy,carrier,model,wifi_ssid,wifi_bssid,geohash
3407,3407,f25b0934-906d-8c68-3758-a7639986c080,2018-04-08 00:05:25,46.163409,-118.947039,7.01,55,Droid,NaN,NaN,c25qt7c3m5
3488,3488,09996809-52d3-4d88-72c6-8bee989757d8,2018-04-08 00:05:33,46.267000,-119.180354,1.96,55,iPhone 4s,NaN,NaN,c25ppyxsb3
4899,4899,8236b855-ef97-9bf2-1792-573c6613565e,2018-04-08 00:07:31,46.164546,-118.948563,1.74,55,Droid Maxx,NaN,NaN,c25qt7brcx
11259,11259,12b4d107-40e0-179a-c43e-993287807944,2018-04-08 00:17:17,46.167459,-118.952640,31.75,55,Moto E,NaN,NaN,c25qtht2fw
11278,11278,09996809-52d3-4d88-72c6-8bee989757d8,2018-04-08 00:17:19,46.251303,-119.087415,4.19,55,iPhone 4s,NaN,NaN,c25r45g1q5
...,...,...,...,...,...,...,...,...,...,...,...
1338029,1338029,57907736-8697-5075-c058-d677439e5eb7,2018-04-08 23:37:53,46.248900,-119.125107,9.88,55,iPhone X,NaN,NaN,c25r15rerg
1338255,1338255,26228874-66e6-89c4-e428-077b8700d973,2018-04-08 23:38:35,46.267359,-119.181800,3.12,55,Lumia 730,NaN,NaN,c25ppywqyu
1338667,1338667,40b90559-2252-0934-3942-8622929f82d1,2018-04-08 23:39:57,46.250595,-119.073887,29.55,55,Droid Maxx,NaN,NaN,c25r47tj1r
1340228,1340228,4a184d31-a827-5608-a2c8-67f4410ac088,2018-04-08 23:45:14,46.249759,-119.095478,7.30,55,Droid,NaN,NaN,c25r1gt265


In [99]:
latitudes, longitudes = zip(*all_corners)

north_boundary = max(latitudes)
south_boundary = min(latitudes)
east_boundary = max(longitudes)
west_boundary = min(longitudes)

In [100]:
nw = (north_boundary, west_boundary)
ne = (north_boundary, east_boundary)
sw = (south_boundary, west_boundary)
se = (south_boundary, east_boundary)